In [1]:
import pandas as pd
import nltk
from keras.models import Sequential
from keras.layers import Dense
import nltk
import re
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

C:\Users\cardo\Documents\Anaconda3\envs\Analise_with_keras\lib\site-packages\sklearn\feature_extraction\text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, defaultdict
Using TensorFlow backend.
C:\Users\cardo\Documents\Anaconda3\envs\Analise_with_keras\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# ----------- Início da Declaração de funções ---------------
def marque_negacao(texto):
    negacoes = ['não','not']
    negacao_detectada = False
    resultado = []
    palavras = texto.split()
    for p in palavras:
        p = p.lower()
        if negacao_detectada == True:
            p = p + '_NEG'
        if p in negacoes:
            negacao_detectada = True
        resultado.append(p)
    return (" ".join(resultado))

def Preprocessing(instancia):
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','').replace('"','')
    instancia = remove_emoji(instancia)
    instancia = Lemmatization(instancia)
    instancia  = Stemming(instancia)
    stopwords = set(nltk.corpus.stopwords.words('english'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

def Lemmatization(instancia):
    wordnet_lemmatizer = WordNetLemmatizer()
    palavras = []
    for w in instancia.split():
        palavras.append(wordnet_lemmatizer.lemmatize(w))
    return (" ".join(palavras))

def Limpeza_dados(instancia):
    # remove links, pontos, virgulas,ponto e virgulas dos tweets
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)

# Aplicando o stemming em nossa base:
def Stemming(instancia):
    stemmer = nltk.stem.snowball.EnglishStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

# Função para remover Stopwords da nossa base:
def RemoveStopWords(instancia):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

# Função para remover os emojis do texto by: https://stackoverflow.com/a/49146722/330558 &
# https://www.reddit.com/r/learnpython/comments/8br5sz/removing_emojis_from_words_python3/dx91wrm/
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               "\U0001F600-\U0001F64F"  # emoticons
                               "\U0001F300-\U0001F5FF"  # symbols & pictographs
                               "\U0001F680-\U0001F6FF"  # transport & map symbols
                               "\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "\U00002702-\U000027B0"
                               "\U000024C2-\U0001F251"
                               "\U00010000-\U0010ffff"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

# Função para mostrar os resultados do classificador
def Metricas(modelo, tweets, classes):
    resultados = cross_val_predict(modelo, tweets, classes, cv=10)
    # Matriz de confusão:
    print (pd.crosstab(classes, resultados, rownames=['Real'], colnames=['Predito'], margins=True))
    print (metrics.classification_report(classes,resultados))
    return 'Acurácia do modelo: {}'.format(metrics.accuracy_score(classes,resultados))

# ----------- Final da Declaração de funções ---------------

In [3]:
df = pd.read_csv('DB/todos_comentarios.csv')

In [4]:
x = df['Comentário']
y = df['Avaliação']

In [5]:
x = [Preprocessing(i) for i in x]

In [29]:
type(x)

pandas.core.series.Series

In [6]:
tweet_tokenizer = TweetTokenizer() 

In [7]:
# Instancia o objeto que faz a vetorização dos dados de texto:
vectorizer = CountVectorizer(analyzer="word", tokenizer=tweet_tokenizer.tokenize)
freq_tweets = vectorizer.fit_transform(x)
len(vectorizer.vocabulary_)

28992

In [8]:
#Rodar essa celula se quiser UNDERsampling
bl = NearMiss()
freq_tweets,classes = bl.fit_sample(freq_tweets, y)
freq_tweets.shape

(15680, 28992)

In [9]:
x_train,x_test,y_train,y_test = train_test_split(freq_tweets,classes)

In [39]:
clf = Sequential(
    [
        Dense(units = 512, activation = 'relu', input_dim = 28992),
        Dense(units = 512, activation = 'relu'),
        Dense(units = 512, activation = 'relu'),
        Dense(units = 512, activation = 'relu'),
        Dense(units = 512, activation = 'relu'),
        Dense(units = 512, activation = 'relu'),
        Dense(units = 512, activation = 'relu'),
        Dense(units = 6, activation = 'softmax'),
    ]
)
clf.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [40]:
clf.fit(x_train,y_train,batch_size=500,epochs = 3)

Epoch 1/3
11760/11760 [==============================] - 214s 18ms/step - loss: 1.4992 - accuracy: 0.2945
Epoch 2/3
11760/11760 [==============================] - 223s 19ms/step - loss: 0.9379 - accuracy: 0.6190
Epoch 3/3
11760/11760 [==============================] - 303s 26ms/step - loss: 0.6098 - accuracy: 0.7704


In [41]:
clf.evaluate(x_test,y_test)

3920/3920 [==============================] - 68s 17ms/step


[1.27520634544139, 0.5237244963645935]

In [ ]:
clf.save('Models/modelo7camadas.h5')

In [31]:
respostas = pd.DataFrame(index = ['predict','text','real'])


ValueError: Error when checking input: expected dense_4_input to have shape (28992,) but got array with shape (9,)

In [ ]:
clf.predict(x_test)